In [1]:
from enum import Enum
import input
import numpy as np
from ast import literal_eval
from IPython.display import clear_output
import time


class TileType(Enum):
  EMPTY = 0
  ROCK = 1
  SAND = 2
  SETTLED_SAND = 3
  
tiletype_to_display = {
    TileType.EMPTY: '.',
    TileType.ROCK: '#',
    TileType.SAND: '+',
    TileType.SETTLED_SAND: 'O'
}

data = [list(map(literal_eval, x.split(' -> '))) for x in input.read_input(14).splitlines()]

In [2]:
flattened = [x for y in data for x in y]
min_y = 0
max_y = max(flattened, key=lambda x: x[1])[1] + 2
min_x = min(flattened, key=lambda x: x[0])[0] - max_y
max_x = max(flattened, key=lambda x: x[0])[0] + max_y

grid = np.full((max_y - min_y + 1, max_x - min_x + 1), TileType.EMPTY, dtype=TileType)
ground_layer = np.full((max_x - min_x + 1), TileType.ROCK, dtype=TileType)
grid[-1] = ground_layer

for segments in data:
    # draw line between points
    for i in range(len(segments) - 1):
        x1, y1 = segments[i]
        x2, y2 = segments[i+1]
        if x1 == x2:
        # vertical line
            for y in range(min(y1, y2), max(y1, y2) + 1):
                grid[y - min_y, x1 - min_x] = TileType.ROCK
        else:
        # horizontal line
            for x in range(min(x1, x2), max(x1, x2) + 1):
                grid[y1 - min_y, x - min_x] = TileType.ROCK

sand_directions = [(0, 1), (-1, 1), (1, 1)]

settled_sand_count = 0
has_sand_settled = False
should_print = False

def get_tile(x, y):
    if x < 0 or x >= grid.shape[1] or y < 0 or y >= grid.shape[0]:
        return TileType.EMPTY
    return grid[y, x]

# simulate sand falling
while not has_sand_settled:
    # place sand at (500, 0)
    if get_tile(500 - min_x, 0) == TileType.EMPTY:
        grid[0, 500 - min_x] = TileType.SAND
    else:
        has_sand_settled = True
        
    sand_locations = [(500 - min_x, 0)]
    
    # simulate sand falling
    while sand_locations:
        for direction in sand_directions:
            dx, dy = direction
            x, y = sand_locations[0]
            
            if x + dx >= 0 and x + dx < grid.shape[1] and y + dy >= 0 and y + dy < grid.shape[0]:
                if grid[y + dy, x + dx] == TileType.EMPTY:
                    grid[y, x] = TileType.EMPTY
                    grid[y + dy, x + dx] = TileType.SAND
                    sand_locations.pop(0)
                    sand_locations.append((x + dx, y + dy))
                    break
            
        # if sand location has not moved, settle it
        if sand_locations and sand_locations[0] == (x, y):
            grid[y, x] = TileType.SETTLED_SAND
            sand_locations.pop(0)
            settled_sand_count += 1

        # clear_output(wait=True)
        # time.sleep(0.1)
        # for y in range(grid.shape[0]):
        #     for x in range(grid.shape[1]):
        #         print(f'{tiletype_to_display[grid[y,x]]}', end=' ', flush=True)
        #     print(flush=True)
            
print(f'Settled sand count: {settled_sand_count - 1}', flush=True) # don't ask about the -1

Settled sand count: 26626
